# Kerchunk Tutorial

I adapt the Kerchunk from `fsspec.github.io` to load files from the NSIDC Bootstrap Sea Ice Concentrations from Nimbus-7 SMMR and DMSP SSM/I-SSMIS, Version 4, dataset.

DATA SET ID: NSIDC-0079
DOI: 10.5067/X5LG68MH013O
Dataset Landing Page: https://nsidc.org/data/nsidc-0079/versions/4#anchor-1

In [1]:
from kerchunk.hdf import SingleHdf5ToZarr
import fsspec

from pathlib import Path
import os
import ujson
import re

import earthaccess

import xarray as xr
import hvplot.xarray

/home/apbarret/mambaforge/envs/test_kerchunk/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def timestamp_from_filename(filepath):
    try:
        match = re.search(r"_(\d{8})_", filepath)
    except re.error as e:
        print(f"Regex search failed: {e.msg}")
    return match.groups(0)

In [18]:
%%time

files = earthaccess.open(result)
ds = xr.open_mfdataset(files, decode_coords='all')
ds

Opening 32 granules, approx size: 0.0 GB


QUEUEING TASKS | : 32it [00:00, 6277.14it/s]
PROCESSING TASKS | : 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:02<00:00, 11.75it/s]
COLLECTING RESULTS | : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 82544.73it/s]


CPU times: user 1.18 s, sys: 60 ms, total: 1.24 s
Wall time: 39.3 s


<xarray.Dataset>
Dimensions:     (time: 31, y: 448, x: 304)
Coordinates:
  * time        (time) datetime64[ns] 2021-03-01 2021-03-02 ... 2021-03-31
  * y           (y) float64 5.838e+06 5.812e+06 ... -5.312e+06 -5.338e+06
  * x           (x) float64 -3.838e+06 -3.812e+06 ... 3.712e+06 3.738e+06
    crs         |S1 b''
Data variables:
    F17_ICECON  (time, y, x) float64 dask.array<chunksize=(1, 448, 304), meta=np.ndarray>
Attributes: (12/49)
    title:                     Bootstrap Sea Ice Concentrations from Nimbus-7...
    summary:                   These sea ice concentration data were derived ...
    id:                        10.5067/X5LG68MH013O
    license:                   Access Constraint: These data are freely, open...
    acknowledgment:            These data are produced by the NASA Cryospheri...
    metadata_link:             https://doi.org/10.5067/X5LG68MH013O
    ...                        ...
    geospatial_y_resolution:   25000.0 meters
    geospatial_lat_min:        30.980564
    geospatial_lat_max:        90.0
    geospatial_lon_min:        -180.0
    geospatial_lon_max:        180.0
    platform:                  DMSP 5D-3/F17 Defense Meteorological Satellite...

In [19]:
ds = ds.where(ds.F17_ICECON <= 1.)

In [20]:
ds.hvplot(groupby='time', width=700, height=700)

:DynamicMap   [time]
   :Image   [x,y]   (F17_ICECON)

In [15]:
ds.F17_ICECON.max().values

array(1.2)

In [9]:
ds.F17_ICECON.encoding

{'chunksizes': (1, 448, 304),
 'fletcher32': False,
 'shuffle': True,
 'preferred_chunks': {'time': 1, 'y': 448, 'x': 304},
 'zlib': True,
 'complevel': 4,
 'source': '<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP1/PM/NSIDC-0079.004/2021.03.01/NSIDC0079_SEAICE_PS_N25km_20210301_v4.0.nc>',
 'original_shape': (1, 448, 304),
 'dtype': dtype('int16'),
 'scale_factor': 0.001,
 'add_offset': 0.0,
 'coordinates': 'time y x',
 'grid_mapping': 'crs'}

In [12]:
!ncdump -h /home/apbarret/Downloads/NSIDC0079_SEAICE_PS_N25km_20230331_v4.0.nc

netcdf NSIDC0079_SEAICE_PS_N25km_20230331_v4.0 {
dimensions:
	time = UNLIMITED ; // (1 currently)
	y = 448 ;
	x = 304 ;
variables:
	double time(time) ;
		time:_FillValue = NaN ;
		time:standard_name = "time" ;
		time:coverage_content_type = "coordinate" ;
		time:units = "days since 1970-01-01 00:00:00" ;
		time:long_name = "ANSI date" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:coverage_content_type = "coordinate" ;
		y:long_name = "y" ;
		y:short_name = "y" ;
		y:axis = "Y" ;
		y:units = "meters" ;
		y:valid_range = -5350000., 5850000. ;
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:coverage_content_type = "coordinate" ;
		x:long_name = "x" ;
		x:short_name = "x" ;
		x:axis = "X" ;
		x:units = "meters" ;
		x:valid_range = -3850000., 3750000. ;
	short F17_ICECON(time, y, x) ;
		F17_ICECON:FillValue = 1100s ;
		F17_ICECON:coverage_content_type = "image" ;
		F17_ICECON:coordinates = "time y x

## Try defining a virtual zarr

In [11]:
#fs = fsspec.filesystem("http")

# flist = (fs.glob("https://n5eil01u.ecs.nsidc.org/PM/NSIDC-0079.004/2023.03.31/NSIDC0079_SEAICE_PS_N25km_*_v4.0.nc"))
flist = session.glob("https://n5eil01u.ecs.nsidc.org/PM/NSIDC-0079.004/2023.03.31/NSIDC0079_SEAICE_PS_N25km_*_v4.0.nc")

fs_local = fsspec.filesystem('') # local filesystem to save final jsons to

# kwargs for fs.open
so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first')  

def gen_json(file_url):
    with session.open(file_url, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, file_url, inline_threshold=300)
        # inline_threshold adjusts the size below which binary blocks are
        # included directly in the output.  A higher threshold results in a larger
        # json file but faster loading time
        variable = "f17_icecon"
        date = timestamp_from_filename(file_url)
        outf = f"{date}_{variable}.json"
        with fs_local.open(outf, "wb") as f:
            print(f"Writing to {outf}")
            f.write(ujson.dumps(h5chunks.translate().encode()))
        

In [12]:
%%time

for file in flist:
    gen_json(file)

FileNotFoundError: https://n5eil01u.ecs.nsidc.org/PM/NSIDC-0079.004/2023.03.31/NSIDC0079_SEAICE_PS_N25km_20230331_v4.0.nc

In [25]:
fs_local.ls('')

['/home/apbarret/src/test_kerchunk/.ipynb_checkpoints',
 '/home/apbarret/src/test_kerchunk/environment.yml',
 '/home/apbarret/src/test_kerchunk/kerchunk_nsidc_bootstrap_sic.ipynb']

In [17]:
%%time

ds = xr.open_dataset("reference://", engine="zarr",
                     backend_kwargs = {
                         "consolidated": False,
                         "storage_options": {
                             "fo": "20230331_f17_icecon.json", 
                             "remote_protocol": "http",
                         }
                     })
print(ds)

FileNotFoundError: [Errno 2] No such file or directory: '/home/apbarret/src/test_kerchunk/20230331_f17_icecon.json/.zmetadata'